# Example usage of MeshFleet Dataset

In [1]:
import pandas as pd

meshfleet_df = pd.read_csv('./data/meshfleet_with_vehicle_categories_df.csv')
meshfleet_df.head()

,sha256,yolo_objects,yolo_objects_confidence,cap3D_caption,trellis_aesthetic_score,trellis_caption,aesthetic_score,fileIdentifier,source,license,fileType,metadata,mesh_filename,caption,avg_height_of_bbox,normalized_height_of_object,normalized_depth_of_object,normalized_width_of_object,normalized_wheelbase,category
0,a0a77f7783b2f0a48d718a0b31fc793d1c16c72beec0d8...,['car'],[0.95],Black Ford Crown Victoria,5.532260,"[""Matte black four-door sedan with a boxy desi...",4.092780,https://sketchfab.com/3d-models/76d7f3373a5548...,sketchfab,Creative Commons - Attribution,glb,{},76d7f3373a5548ef86821c95c68fbe55.glb,The images depict a three-dimensional renderin...,131.5000,1.0,3.592000,1.473485,2.050560,Sedan
1,4f0e8fe23f6bdc119ee773c3e7f796e90ee41fed1066be...,['car'],[0.88],Blue Volkswagen Beetle,6.863236,"[""A classic Volkswagen Beetle with a rounded b...",4.495291,https://sketchfab.com/3d-models/66d536cb479944...,sketchfab,Creative Commons - Attribution,glb,{},66d536cb4799441bab6270d569d32c42.glb,The car depicted in the images is a classic Vo...,179.8125,1.0,2.593567,1.174515,1.611789,Coupe
2,5a60c7a3e95d1616d84d677ab065fd1220788f4833eed5...,['car'],[0.95],a Dodge Charger SXT,5.771935,"[""A modern red sedan with black trims, slim he...",4.033427,https://sketchfab.com/3d-models/d72548ce431c45...,sketchfab,Creative Commons - Attribution,glb,{},d72548ce431c45809418af01ee93c096.glb,The 3D car renderings showcase a sleek sedan w...,147.6250,1.0,3.210145,1.391304,2.097391,Sedan
3,5b126f4af8a10c06c3bbe1a9b1998eb395b9072c278b3f...,"['car', 'truck', 'bus']","[0.95, 0.75, 0.39]",Small yellow car model,6.822062,"[""A yellow compact hatchback car with a stream...",4.038136,https://sketchfab.com/3d-models/f57b9138ff9645...,sketchfab,Creative Commons - Attribution,glb,{},f57b9138ff96450e8d359304b15fc3de.glb,The car rendered in the images appears to be a...,185.5625,1.0,2.548571,1.345845,1.765851,Hatchback
4,654e96a4a1fb30c8f13ad4e692c85cdb6826088a1b3eb7...,['car'],[0.96],a silver sports car.,6.064985,"[""A sleek, classic sports car with a low, aero...",4.256062,https://sketchfab.com/3d-models/2e8630e7c08546...,sketchfab,Creative Commons - Attribution,glb,{},2e8630e7c085464cb332cec0f5faaef0.glb,"The car depicted in the images is a stylized, ...",138.1875,1.0,3.404580,1.620567,2.057771,Sports Car


In [2]:
only_hatchback_df = meshfleet_df[meshfleet_df['category'] == 'Hatchback']
len(only_hatchback_df)

103

In [3]:
average_length_of_hatchback = only_hatchback_df['normalized_depth_of_object'].mean()
average_width_of_hatchback = only_hatchback_df['normalized_width_of_object'].mean()
average_wheelbase_of_hatchback = only_hatchback_df['normalized_wheelbase'].mean()
print(f'Average length of hatchback: {average_length_of_hatchback}')
print(f'Average width of hatchback: {average_width_of_hatchback}')
print(f'Average wheelbase of hatchback: {average_wheelbase_of_hatchback}')

Average length of hatchback: 2.677314238751248
Average width of hatchback: 1.32098296277762
Average wheelbase of hatchback: 1.7440673484721436


In [4]:
long_hatchback_df = only_hatchback_df[only_hatchback_df['normalized_depth_of_object'] > average_length_of_hatchback + 0.5]
len(long_hatchback_df)

12

## Loading MeshFleet Objects from Objaverse XL

You can use the annotations from objaverse, and filter by the selected sha256:

```python
annotations_df = oxl.get_annotations(download_dir="~/.objaverse")
annotations_to_download = annotations_df[annotations_df['sha256'].isin(long_hatchback_df['sha256'].tolist())]
annotations_to_download.head()
```

You can also directly use the annotations from the meshfleet dataset:


In [5]:
import os 
import shutil
import objaverse.xl as oxl
from typing import Any, Dict, Hashable

base_dir = "./downloads"
g_base_path = os.path.abspath(base_dir)
os.makedirs(g_base_path, exist_ok=True)

def handle_found_object(
    local_path: str,
    file_identifier: str,
    sha256: str,
    metadata: Dict[Hashable, Any]
) -> None:
    # move the file to the shape folder
    filename = os.path.basename(local_path)
    shape_path = os.path.join(g_base_path, "shape")
    os.makedirs(shape_path, exist_ok=True)
    new_file_name = f"{sha256}.{filename.split('.')[-1]}"
    new_file_path = os.path.join(shape_path, new_file_name)
    shutil.copy2(local_path, new_file_path)

In [7]:
n_processes = 4
oxl.download_objects(
    objects=long_hatchback_df,
    processes=n_processes,
    handle_found_object=handle_found_object
)

2025-03-05 18:53:19.283 | INFO     | objaverse.xl.github:download_objects:602 - Provided 5 repoIds with 6 objects to process.
2025-03-05 18:53:19.284 | INFO     | objaverse.xl.github:download_objects:614 - Found 5 repoIds not yet downloaded. Downloading now...


Handling 3D object files: 100%|██████████| 63/63 [00:00<00:00, 78.56it/s] 
2025-03-05 18:53:46.552 | ERROR    | objaverse.xl.github:_run_command_with_check:173 - None
2025-03-05 18:53:46.554 | ERROR    | objaverse.xl.github:_run_command_with_check:174 - None
2025-03-05 18:53:46.557 | ERROR    | objaverse.xl.github:_process_repo:217 - Could not clone ShovitJha/Biomed-endless-runner
2025-03-05 18:54:05.576 | INFO     | objaverse.xl.sketchfab:download_objects:508 - Found 6 objects already downloaded
2025-03-05 18:54:05.577 | INFO     | objaverse.xl.sketchfab:download_objects:529 - Downloading 0 new objects across 4 processes


{'https://sketchfab.com/3d-models/fe02fba6302e450ea8424591493341ea': '~/.objaverse/hf-objaverse-v1/glbs/000-023/fe02fba6302e450ea8424591493341ea.glb',
 'https://sketchfab.com/3d-models/1532cb9361bf459fb7a57672dcfcda11': '~/.objaverse/hf-objaverse-v1/glbs/000-044/1532cb9361bf459fb7a57672dcfcda11.glb',
 'https://sketchfab.com/3d-models/6dd512527a0345b09bc01911b9928062': '~/.objaverse/hf-objaverse-v1/glbs/000-136/6dd512527a0345b09bc01911b9928062.glb',
 'https://sketchfab.com/3d-models/2e14302c06b546b0821af441da718b68': '~/.objaverse/hf-objaverse-v1/glbs/000-004/2e14302c06b546b0821af441da718b68.glb',
 'https://sketchfab.com/3d-models/cfe08291b94f4b998c46b86bb7e85096': '~/.objaverse/hf-objaverse-v1/glbs/000-143/cfe08291b94f4b998c46b86bb7e85096.glb',
 'https://sketchfab.com/3d-models/87e2581386404aaaa08226b45abed511': '~/.objaverse/hf-objaverse-v1/glbs/000-064/87e2581386404aaaa08226b45abed511.glb'}